In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [51]:
census_tracts_philly = pd.read_csv("./data/Census_Tract_groups_2010.txt", sep='\t', header=0)

# new data frame with split value columns

splits = census_tracts_philly["Geolocation"].str.split(",", n = 1, expand = True)

# reformat geo location string
census_tracts_philly["lat"] = splits[0].str.split("(", n = 1, expand = True)[1]
census_tracts_philly["lon"] = splits[1].str.split(")", n = 1, expand = True)[0]

census_tracts_philly

,TractFIPS,Geolocation,lat,lon
0,42101000100,"(39.95238276040, -75.1466628627)",39.95238276040,-75.1466628627
1,42101000200,"(39.95539990180, -75.1569774999)",39.95539990180,-75.1569774999
2,42101000300,"(39.95710584440, -75.1709368581)",39.95710584440,-75.1709368581
3,42101000401,"(39.95413660830, -75.1765601839)",39.95413660830,-75.1765601839
4,42101000402,"(39.95329732390, -75.1686952355)",39.95329732390,-75.1686952355
...,...,...,...,...
371,42101039000,"(40.03573024550, -75.0938838844)",40.03573024550,-75.0938838844
372,42101980000,"(39.98504729620, -75.2002069117)",39.98504729620,-75.2002069117
373,42101980100,"(40.04704935510, -75.2119391455)",40.04704935510,-75.2119391455
374,42101980200,"(40.06981173210, -75.0490217180)",40.06981173210,-75.0490217180


In [36]:
 headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}

In [37]:
def extract_score(current_string, score_type):
    # initializing substrings
    sub1 = "="
    sub2 = score_type

    # getting index of substrings
    idx1 = current_string.find(sub1)
    idx2 = current_string.find(sub2) -1

    # length of substring 1 is added to
    # get string from next character
    res = current_string[idx1 + len(sub1) + 1: idx2]
    
    return((res))
def get_redfin_data(row):
    url = "https://www.walkscore.com/score/loc/lat=" + str(row['lat'])+ "/lng=" + str(row['lon'])[1:]
    print(str(row["TractFIPS"]) + " : ("+ str(row['lat'])+ "," + str(row['lon']) + ")")
    
    response=requests.get(url,headers=headers)
    doc = BeautifulSoup(response.text,'html.parser')
    
    scores = doc.find_all('img')
    
    return(pd.Series([extract_score(str(scores[3]), "Walk"),
            extract_score(str(scores[4]), "Transit"),
            extract_score(str(scores[5]), "Bike")]))

redfin_scores = census_tracts_philly.apply(get_redfin_data, 1)

42101000100 : (39.95238276040, -75.1466628627)
42101000200 : (39.95539990180, -75.1569774999)
42101000300 : (39.95710584440, -75.1709368581)
42101000401 : (39.95413660830, -75.1765601839)
42101000402 : (39.95329732390, -75.1686952355)
42101000500 : (39.95195339670, -75.1581776292)
42101000600 : (39.94938447630, -75.1583354851)
42101000700 : (39.95117366440, -75.1727351974)
42101000801 : (39.95005996660, -75.1800508587)
42101000803 : (39.94937478010, -75.1742488768)
42101000804 : (39.94856456540, -75.1677563238)
42101000901 : (39.94788664450, -75.1623004239)
42101000902 : (39.94715954610, -75.1566388733)
42101001001 : (39.94567226110, -75.1516019404)
42101001002 : (39.94500453800, -75.1466193030)
42101001101 : (39.94499176360, -75.1625240137)
42101001102 : (39.94426542970, -75.1566960284)
42101001201 : (39.94713672810, -75.1802279195)
42101001202 : (39.94591163600, -75.1699475320)
42101001300 : (39.94283179630, -75.1845558386)
42101001400 : (39.94294360200, -75.1715930244)
42101001500 :

42101017900 : (39.98705345300, -75.1139338890)
42101018001 : (39.98105698160, -75.1128000114)
42101018002 : (39.98363441910, -75.1067095894)
42101018300 : (39.99526020690, -75.0746013479)
42101018400 : (39.99869535850, -75.0645677689)
42101018800 : (39.99736586560, -75.1058294984)
42101019000 : (40.00924797460, -75.0985073036)
42101019100 : (40.00967460390, -75.1068374251)
42101019200 : (40.00063923830, -75.1150647872)
42101019501 : (40.00043024170, -75.1316227229)
42101019502 : (40.00340292780, -75.1300444941)
42101019700 : (40.01627991660, -75.1379372140)
42101019800 : (40.01072447060, -75.1421472184)
42101019900 : (40.00485156110, -75.1435462441)
42101020000 : (40.00252894560, -75.1508995206)
42101020101 : (40.00368246810, -75.1553071488)
42101020102 : (40.01114158900, -75.1545675851)
42101020200 : (40.00812824590, -75.1621607972)
42101020300 : (40.01258211350, -75.1495922294)
42101020400 : (40.01982161430, -75.1525117069)
42101020500 : (40.01587384890, -75.1643605132)
42101020600 :

42101036502 : (40.12357054570, -74.9993519096)
42101036600 : (39.94486130080, -75.1389504927)
42101036700 : (39.96072487830, -75.1439885049)
42101036900 : (39.94909581550, -75.1937044052)
42101037200 : (39.91294851030, -75.1599380729)
42101037300 : (39.90543391290, -75.1837391301)
42101037500 : (39.98434936590, -75.2543798218)
42101037600 : (39.95990085750, -75.1551650837)
42101037700 : (39.98243813490, -75.1506932327)
42101037800 : (39.97956898930, -75.0949503216)
42101037900 : (39.99108998160, -75.0927153152)
42101038000 : (40.00522790100, -75.0807842620)
42101038100 : (40.01769739120, -75.0396349998)
42101038200 : (39.99745810900, -75.0974658211)
42101038300 : (40.01086314620, -75.1248985484)
42101038400 : (40.07141662110, -75.2332568528)
42101038500 : (40.07590534380, -75.2136899940)
42101038600 : (40.05805199480, -75.2102737755)
42101038700 : (40.08329712590, -75.2143439224)
42101038800 : (40.05773067780, -75.1967866797)
42101038900 : (40.05434313720, -75.1682763546)
42101039000 :

In [52]:
census_tracts_philly["walk_score"] = redfin_scores[0]
census_tracts_philly["transit_score"] = redfin_scores[1]
census_tracts_philly["bike_score"] = redfin_scores[2]

In [61]:
# some of the scores are off, need to write code for test cases
# incase a location doesn't have a transit score=

In [62]:
census_tracts_philly.to_csv("./data/walkscores.csv", sep=',', na_rep='', float_format=None, header=True, index=False)